# Reference

[inplace_abn/modules/deeplab.py](https://github.com/mapillary/inplace_abn/blob/master/modules/deeplab.py)

[inplace_abn/modules/bn.py](https://github.com/mapillary/inplace_abn/blob/master/modules/bn.py)

[inplace_abn/modules/functions.py](https://github.com/mapillary/inplace_abn/blob/master/modules/functions.py)

[Inplace ABN](https://arxiv.org/abs/1712.02616)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# BN

In [34]:
class ABN(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True, activation="leaky_relu", slope=0.01):
        super().__init__()
        self.num_features = num_features
        self.affine = affine
        self.eps = eps
        self.momentum = momentum
        self.activation = activation
        self.slope = slope
        if self.affine:
            self.weight = nn.Parameter(torch.ones(num_features))
            self.bias = nn.Parameter(torch.zeros(num_features))
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        self.reset_parameters()
    def reset_parameters(self):
        nn.init.constant_(self.running_mean, 0)
        nn.init.constant_(self.running_var, 1)   
        if self.affine:
            nn.init.constant_(self.weight, 1)
            nn.init.constant_(self.bias, 0)
    def forward(self, x):
        x = F.batch_norm(x, self.running_mean, self.running_var, weight=self.weight, bias=self.bias,
                         training=self.training, momentum=self.momentum, eps=self.eps)
        
        if self.activation == ACT_RELU:
            return F.relu(x, inplace=True)
        elif self.activation == ACT_LEAKY_RELU:
            return F.leaky_relu(x, negative_slope=self.slope, inplace=True)
        elif self.activation == ACT_ELU:
            return F.elu(x, inplace=True)
        else:
            return x
    

In [41]:
class InplaceABN(ABN):
    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True, activation="leaky_relu", slope=0.01):
        super(InplaceABN, self).__init__(num_features, eps, momentum, affine, activation, slope)
    
    def forward(self ,x):
        return 
        

In [43]:
z = InplaceABN(10)
print(z)

InplaceABN()


In [52]:
z = torch.randn((5,3,2,10))
z = z.new_empty(20)
print(z.shape)
print(z)

torch.Size([20])
tensor([-2.4721e+31,  2.8671e-42,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00, -2.5027e+31,  2.8671e-42,  0.0000e+00,  0.0000e+00,
        -3.6541e-09,  2.8671e-42,  0.0000e+00,  0.0000e+00,         nan,
         6.8664e-44,  1.4013e-45,  0.0000e+00,  1.4013e-45,  5.7453e-44])


# DeepLab

In [ ]:
class DeepLabV3(nn.Module):
    def __init__(self,
                 in_channels,
                 hidden_channels=256,
                 out_channels,
                 dilations=(12,24,36),
                 norm_act = ABN,
                 pooling_size=None):
        super().__init__()
        
        self.pooling_size = pooling_size
        self.map_convs = nn.ModuleList([
            
            nn.Conv2(in_channels, hidden_channels, kernel_size=1, bias=False),
            
            nn.Conv2(in_channels, hidden_channels, kernel_size=3, padding=dilations[0], dilation=dilations[0], bias=False),
            nn.Conv2(in_channels, hidden_channels, kernel_size=3, padding=dilations[1], dilation=dilations[1], bias=False),
            nn.Conv2(in_channels, hidden_channels, kernel_size=3, padding=dilations[2], dilation=dilations[2], bias=False)
            # padding size = (k-1)*d/2
        ])
        
        self.map_bn = norm_act(hidden_channels * 4)
        
        self.conv1x1 = nn.Conv2d(hidden_channels * 4, out_channels, kernel_size=1, bias=False)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        self.global_pooling_conv = nn.Conv2d(in_channels, hidden_channels, kernel_size=1, bias=False)
        self.global_pooling_bn = norm_act(hidden_channels)
        self.pool_conv1x1 = nn.Conv2d(hidden_channels, out_channels, kernel_size=1, bias=False)
        
        
        self.reduced_bn = norm_act(out_channels)
        
    def forward(self, x):
        out = torch.cat([m(x) for m in self.map_convs], dim=1)
        out = self.map_bn(out)
        out = self.conv1x1(out)
        
        # global pooling
        pool = self._global_pooling(x)
        pool = self.global_pooling_conv(pool)
        pool = self.global_pooling_bn(pool)
        pool = self.pool_red_conv(pool)
        pool = self.pool_conv1x1(pool)
    
         if self.training or self.pooling_size is None:
            pool = pool.repeat(1, 1, x.size(2), x.size(3))

        out += pool
        out = self.self.reduced_bn(out)
        return out
    
    
    def _global_pooling(self, x):
        if self.training or self.pooling_size is None:
            pool = self.avg_pool(x)
        else:
            pooling_size = (min(self.pooling_size[0], x.size(2)),
                            min(self.pooling_size[1], x.size(3)))
            
            padding = (
                (pooling_size[1] - 1) // 2,
                (pooling_size[1] - 1) // 2 if pooling_size[1] % 2 == 1 else (pooling_size[1] - 1) // 2 + 1,
                (pooling_size[0] - 1) // 2,
                (pooling_size[0] - 1) // 2 if pooling_size[0] % 2 == 1 else (pooling_size[0] - 1) // 2 + 1
            )
            
            pool = F.avg_pool2d(x pooling_size, stride=1)
            pool = F.pad(pool, pad=padding, mode='replicate')
            
        return pool

In [12]:
x = torch.randn((1,3,2,2))
print(x.shape)
z = x.repeat((1,1,5,5))
print(z.shape)

torch.Size([1, 3, 2, 2])
torch.Size([1, 3, 10, 10])


In [7]:
m = nn.AdaptiveAvgPool2d(1)
z = m(x)
print(z.shape)

torch.Size([1, 3, 1, 1])


In [9]:
a = (5,3)
print(a[0])

5


![](http://liangchiehchen.com/fig/deeplabv3plus.png)